## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

### Задание 1.

In [ ]:
"""
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
Эти слова определяем в начале кода в переменной.

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
"""

In [54]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import json

In [14]:
KEYWORDS = ['python', 'парсинг']

In [48]:
# Перебираем все страницы по ключевым словам и передаем статьи в датафрейм
def get_df_from_all_page(keywords, amount_page):
    df = pd.DataFrame()
    for i in range(1, amount_page + 1):
        url = 'https://m.habr.com/ru/all/'
        if i > 1:
            url = url + 'page' + str(i) + '/'
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')
        all_articles = soup.find_all('div', class_='tm-article-snippet')
        df = make_df(df, all_articles, keywords)
    return df

In [49]:
#Перебираем статьи на отдельной странице и добавляем их в датафрейм 
def make_df(df, all, keywords):
    for i in all:
        title = i.find('a', class_='tm-article-snippet__title-link').get_text()
        if get_need_articles(title, keywords):
            link = "https://m.habr.com" + i.find('a', class_='tm-article-snippet__title-link').get('href')
            date = pd.to_datetime(i.find('time').get('title'))
            row = {'date': date, 'title': title, 'link': link}
            df = pd.concat([df, pd.DataFrame([row])])
    return df

In [50]:
# Получаем текст заголовка и ищем в нем переданные значения
def get_need_articles(title, keywords):
    for i in keywords:
        if i in title.lower():
            return True
    return False

In [51]:
get_df_from_all_page(KEYWORDS, 50)

,date,title,link
0,2021-04-18 13:31:00,Превращаем клавиатуру в пианино с помощью python,https://m.habr.com/ru/post/553032/
0,2021-04-16 00:00:00,Прыжок до небес: запускаем телеграм бота на Py...,https://m.habr.com/ru/post/550456/
0,2021-04-15 11:20:00,Как ускорить код на Python в тысячу раз,https://m.habr.com/ru/company/vdsina/blog/552378/
0,2021-04-14 13:41:00,Регулярные выражения Python для новичков: что ...,https://m.habr.com/ru/company/skillbox/blog/55...
0,2021-04-14 09:50:00,ModulationPy: цифровые схемы модуляции на язык...,https://m.habr.com/ru/post/552242/
0,2021-04-13 20:25:00,Шифрование сообщений в Python. От простого к с...,https://m.habr.com/ru/post/552212/
0,2021-04-12 19:07:00,Телеграмм-бот на Python,https://m.habr.com/ru/post/552052/
0,2021-04-12 16:05:00,Книга «Современный скрапинг веб-сайтов с помощ...,https://m.habr.com/ru/company/piter/blog/551994/
0,2021-04-12 16:03:00,«За что вы так меня не любите?» (с) Python,https://m.habr.com/ru/post/552006/
0,2021-04-12 15:56:00,Как считывать и удалять метаданные из ваших фо...,https://m.habr.com/ru/company/skillfactory/blo...


### Задание 2.

In [ ]:
"""
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 
Список email-ов задаем переменной в начале кода.

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
"""

In [161]:
#почты для проверки
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [162]:
newsfeed = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [163]:
params = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0'
    }

In [164]:
# датафрейм с данными проверки
mail_check = pd.DataFrame(columns = {'email','date','source','description'})

In [165]:
# проверяем емейлы на утечку данных
for email in EMAIL:
    res = requests.post(newsfeed, json={'emailAddresses': [email]},headers = params)
    extracted_data = json.loads(res.text) 
    if extracted_data:
        for key in extracted_data['breaches'].keys():
            description = extracted_data['breaches'][key]['description']
            date = extracted_data['breaches'][key]['publishDate']
            source = extracted_data['breaches'][key]['site']
            site = extracted_data['breaches'][key]['site']
            data_sample = {'email':email,'date':date,'source':source,'description':description}
            mail_check = mail_check.append(data_sample,ignore_index=True) 

In [166]:
mail_check

,source,email,date,description
0,parapa.mail.ru,xxx@x.ru,2017-02-14T00:00:00Z,"In July and August 2016, two criminals execute..."
1,vk.com,xxx@x.ru,2016-10-29T00:00:00Z,Popular Russian social networking platform VKo...
2,adobe.com,xxx@x.ru,2016-10-21T00:00:00Z,"In October of 2013, criminals penetrated Adobe..."
3,cfire.mail.ru,xxx@x.ru,2017-02-14T00:00:00Z,"In July and August of 2016, two criminals carr..."
4,cdprojektred.com,xxx@x.ru,2017-01-31T00:00:00Z,"In March 2016, CDProjektRed.com.com's forum da..."
5,imesh.com,xxx@x.ru,2016-10-23T00:00:00Z,"In June 2016, a cache of over 51 million user ..."
6,azcentral.com,yyy@y.com,2020-01-03T00:00:00Z,"At an unconfirmed date, online Arizona newspap..."
7,wishbone.io,yyy@y.com,2020-05-28T00:00:00Z,"In January 2020, the online poll website Wishb..."
8,myheritage.com,yyy@y.com,2017-11-04T00:00:00Z,"In October 2017, a customer database belonging..."
9,forums.vkmonline.com,yyy@y.com,2021-02-11T00:00:00Z,"At an unconfirmed date, the Russian-language m..."


### Дополнительная часть

In [ ]:
"""
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>
"""

In [160]:
# указываем личный токен
token = 'secret'

In [157]:
# указываем метод, версию API и ссылку на сообщество в ВК
url_method = 'https://api.vk.com/method/wall.get'
version = '5.130'
group = 'netology'

In [158]:
# получаем 50 постов из группы в ВК
def vk_group_post(url_method, token, version, group):
    params = {
        'access_token': token,
        'v': version,
        'count': 50,
        'domain': group
    }
    res = json.loads(requests.get(url_method, params).text)
    df = pd.DataFrame()
    for i in res['response']['items']:
        date = pd.to_datetime(i['date'], unit='s')
        row = {'date' : date, 'text': i['text']}
        df = pd.concat([df, pd.DataFrame([row])])
    return df

In [159]:
vk_group_post(url_method, token, version, group)

,date,text
0,2021-04-14 13:20:03,А у нас конкурс 🎁\n\nРазыгрываем сразу 10 приз...
0,2021-04-18 11:24:00,"Продать помидор под видом сыра, сделать из мух..."
0,2021-04-17 12:01:00,"Рейтинг SEO-сервисов, стрим о цифровизации в п..."
0,2021-04-17 09:01:00,Как часто в три часа ночи вы скроллите рекомен...
0,2021-04-16 14:14:00,*партнёрский пост* \n\nУчитесь в приложении и ...
0,2021-04-16 08:09:00,Собрали подборку бесплатных занятий по копирай...
0,2021-04-15 14:49:00,"Казалось бы, нет офиса — нет проблем 🙃 Но на у..."
0,2021-04-15 07:41:00,"Вот такие новости: новые голоса у Siri, исполь..."
0,2021-04-14 07:44:00,"Мы верим, что в Data Science может работать ка..."
0,2021-04-13 14:49:00,"Excel — как мозг, который мы используем только..."
